In [1]:
#Create the same table as previous notebook
import pandas as pd
df = pd.DataFrame(data = [['Jerry', 45, 0, 'gmail', 'Tim'],
                          ['Terry', 5, 1, 'gmail', 'Buddy'],
                          ['Mary', 30, 1, 'yahoo', 'Cooper'],
                          ['Larry', 45, 0, 'MSN', 'Buddy'],
                          ['Gary', 10, 1, 'MSN', 'Slappy'],
                          ['Larry', 45, 0, 'MSN', 'Buddy']
                         ], 
                  columns = ['Person', 'Age', 'HappyFlag', 'Email', 'DogName'])

## Where Clause
### Limitting the number of rows selected
Now that we have the tools needed to create quick tables, view them, and get the columns we want; we can try out selecting only rows that we want. The first thing to explore is how we might select people from the table who have a particular email.

```sql
Select * from df where Email = 'gmail'
```

What would happen if just selected the singular email column and looked for those values

In [2]:
df['Email'] = 'gmail'

In [3]:
df

,Person,Age,HappyFlag,Email,DogName
0,Jerry,45,0,gmail,Tim
1,Terry,5,1,gmail,Buddy
2,Mary,30,1,gmail,Cooper
3,Larry,45,0,gmail,Buddy
4,Gary,10,1,gmail,Slappy
5,Larry,45,0,gmail,Buddy


Whoops. It looks like we set the entire column to gmail rather than just selecting rows. Python uses the conditional equals == that you might see in object oriented languages. Let’s reset our DataFrame by copying what's above

In [2]:
df = pd.DataFrame(data = [['Jerry', 45, 0, 'gmail', 'Tim'],
                          ['Terry', 5, 1, 'gmail', 'Buddy'],
                          ['Mary', 30, 1, 'yahoo', 'Cooper'],
                          ['Larry', 45, 0, 'MSN', 'Buddy'],
                          ['Gary', 10, 1, 'MSN', 'Slappy'],
                          ['Larry', 45, 0, 'MSN', 'Buddy']], 
                  columns = ['Person', 'Age', 'HappyFlag', 'Email', 'DogName'])

In [3]:
df

,Person,Age,HappyFlag,Email,DogName
0,Jerry,45,0,gmail,Tim
1,Terry,5,1,gmail,Buddy
2,Mary,30,1,yahoo,Cooper
3,Larry,45,0,MSN,Buddy
4,Gary,10,1,MSN,Slappy
5,Larry,45,0,MSN,Buddy


And let's try again with using ==

In [4]:
df['Email'] == 'gmail'

0     True
1     True
2    False
3    False
4    False
5    False
Name: Email, dtype: bool

What we are looking at here is a series of booleans that tell us on which rows our condition is true. What we have to do is take that and use it to define the subset of values we want to see, the way that is accomplished is by placing that series inside an indexer.

In [5]:
df[df['Email'] == 'gmail']

,Person,Age,HappyFlag,Email,DogName
0,Jerry,45,0,gmail,Tim
1,Terry,5,1,gmail,Buddy


Note that what we have done is added the appropriate where clause, the sql might look like
```sql
Select * from df where Email = 'gmail'
```
Let's now combine both the condition in the where clause with what we learned in the previous notebook with selecting a particular group of columns
```sql
Select Person, Age, Email from df where Email = 'gmail'
```

In [6]:
df[df['Email'] == 'gmail'][['Person', 'Age', 'Email']]

,Person,Age,Email
0,Jerry,45,gmail
1,Terry,5,gmail


We have used multiple indexers.
```python
df[Indexer1][Indexer2]
```

Note that it doesn't matter which comes first:

In [7]:
df[['Person', 'Age', 'Email']][df['Email'] == 'gmail']

,Person,Age,Email
0,Jerry,45,gmail
1,Terry,5,gmail


However you should be slightly careful. Notice what happens when I tack on an extra condition

In [8]:
df[['Person', 'Age', 'Email']][df['Email'] == 'gmail'][df['Age'] <= 10]

C:\Users\s.boyer1\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,Person,Age,Email
1,Terry,5,gmail


Results are returned but we get a warning. What is happening here is that the last condition 
```python
df['Age'] <= 10
```
is calculating on a dataframe that doesn't match the original size of df (the data frame with just 2 rows after the first condition of email = gmail is calculated).  At this point what we are essentially looking at is expanding our SQL where condition
```sql
Select Person, Age, Email from df where Email = 'gmail' and Age <= 10 
```
The way I recommend handling this is to use one indexer for the column selection and one and only one indexer for the where conditions. this way we don't run into the warning above. In python we use '&' and '|' to represent 'and' and 'or'. With this we can fit muliple conditions into a singular indexer.

Read thru the following SQL queries and how we would filter the data frame in order to achieve the equivalent results. As with SQL it is always best to explicitly group order of operations of ands and ors using parantheses.  
```sql
Select * from df where Email = 'gmail' and Age <= 10 
```

In [9]:
df[(df['Email'] == 'gmail') & (df['Age'] <= 10)]

,Person,Age,HappyFlag,Email,DogName
1,Terry,5,1,gmail,Buddy


```sql
Select Person, Age, Email from df where Email = 'gmail' and Age <= 10 
```

In [11]:
df[(df['Email'] == 'gmail') & (df['Age'] <= 10)][['Person', 'Age', 'Email']]

,Person,Age,Email
1,Terry,5,gmail


```sql
Select Person, Age, Email from df where (Email = 'gmail' or Email = 'MSN') and Age <= 10 
```

In [12]:
df[((df['Email'] == 'gmail') | (df['Email'] == 'MSN')) & (df['Age'] <= 10)][['Person', 'Age', 'Email']]

,Person,Age,Email
1,Terry,5,gmail
4,Gary,10,MSN


At this point things can be getting kind of tricky to read and keep track of all in one line. When that happens while working with SQL, I know that I can break things into steps and leverage additional temp tables. Taking the last example, we could break it into steps in SQL using a select into and quickly have a smaller set to work with. 
```sql
Select Person, Age, Email 
into #smallerTable
from df 
where (Email = 'gmail' or Email = 'MSN')

Select * from #smallerTable where Age <=10
```
This is just an example and everyone has a different comfortability level when it comes to abstraction. Some people can figure out the most complex of tasks and boil it down into one step. That may make it difficult for others to follow along when reading code. 

This can get a little bit tricky. I've known SQL developers that would never Select * into, and would always write a create table first. In dealing with temp tables that we are just using to test out how syntax works I don't think it is a big deal. Luckly python does make it quite easy for us to achieve the same result


In [13]:
#Define our smaller 'Temp' table
df_smallerTable = df[(df['Email'] == 'gmail') | (df['Email'] == 'MSN')][['Person', 'Age', 'Email']]

#Select * from maller table with additional condition
df_smallerTable[df_smallerTable['Age'] <= 10]

,Person,Age,Email
1,Terry,5,gmail
4,Gary,10,MSN


#### Loc & iLoc

Using Nested data frames this way is not the best way. There are two function of Data frames that were designed to handle this a bit more elegantly

```python
df[((df['Email'] == 'gmail') | (df['Email'] == 'MSN')) & (df['Age'] <= 10)][['Person', 'Age', 'Email']]
```

df.loc[] and df.iloc[] are indexers that have a couple advantages over using just bracket indexers. A good read if you are interested is this [stack overflow thread](https://stackoverflow.com/questions/38886080/python-pandas-series-why-use-loc)

Generally, you can select data with lock the same way we were selecting data with brackets, the conditions for what rows you want to select to directly inside the brackets, but we can use a comma after the conditions to specify which columns we want returned so that we do not need to open a new set of brackets.


In [14]:
df.loc[((df['Email'] == 'gmail') | (df['Email'] == 'MSN')) & (df['Age'] <= 10), ['Person', 'Age', 'Email']]

,Person,Age,Email
1,Terry,5,gmail
4,Gary,10,MSN


As partially noted in the stack overflow thread above, the reason to use the loc function is that we gain added functionality when the index is something meaningful. Specifically we can use ":" to slice the data. Additionally as we will see in the "Updates" section updating specific values in a data frame works with the loc function.

From this point on all selects will be done with the loc (or iloc) indexer.  We will see more of its power when we look at choosing an index.